In [1]:
from docarray import Document, DocumentArray
from jina import Executor, Flow, requests

In [2]:
class PreProcImg(Executor):
    @requests
    async def foo(self, docs: DocumentArray, **kwargs):
        for d in docs:
            (
                d.load_uri_to_image_tensor(200, 200)
                .set_image_tensor_normalization()
                .set_image_tensor_channel_axis(
                    -1, 0
                )
            )

In [3]:
class EmbedImg(Executor):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        import torchvision
        self.model = torchvision.models.resnet50(pretrained=True)


In [4]:
class MatchImg(Executor):
    _da = DocumentArray()

    @requests(on='/index')
    async def index(self, docs: DocumentArray, **kwargs):
        self._da.extend(docs)
        docs.clear()

    @requests(on='/search')
    async def foo(self, docs: DocumentArray, **kwargs):
        docs.match(self._da, limit=9)
        del docs[...][:, ('embeding', 'tensor')]

In [5]:
f = (
    Flow(port=12345)
    .add(uses=PreProcImg)
    .add(uses=EmbedImg, replicas=3)
    .add(uses=MatchImg)
)

In [6]:
index_data = DocumentArray.pull('demo-leftda', show_progress=True)

UserWarning: install "ipywidgets" for Jupyter support (raised from 
/home/marcelo/miniconda3/envs/jina/lib/python3.8/site-packages/rich/live.py:221)

In [7]:
with f:
    f.post(
        '/index',
        index_data,
        show_progress=True,
        request_size=8
    )
    f.block()

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/marcelo/.cache/torch/checkpoints/resnet50-19c8e357.pth
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/marcelo/.cache/torch/checkpoints/resnet50-19c8e357.pth
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/marcelo/.cache/torch/checkpoints/resnet50-19c8e357.pth
7.9%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

24.7%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_r

🔗         Protocol                  GRPC   
 🏠     Local access         0.0.0.0:12345   
 🔒  Private network   192.168.0.101:12345   
 🌐   Public address  88.130.155.133:12345

In [ ]:
from jina import Client

c = Client(port=12345)
print(c.post('/search', index_data[0]['@m']))

In [ ]:
with f:
    ...
    f.protocol = 'http'
    f.block()